In [ ]:
# Download model from remote

import os
os.makedirs('./bin/models', exist_ok=True)
os.makedirs('./bin/_tmp_models', exist_ok=True)

! scp vast4090:/root/sennet-kaggle/src/exp2_small_net/bin/models/unet3d.pt ./bin/models/
! scp vast4090:/root/sennet-kaggle/src/exp2_small_net/bin/_tmp_models/wide_patcher_finder_IN_PROGRESS.pt ./bin/_tmp_models/

ssh: connect to host vast4090 port 40583: Connection refused
ssh: connect to host vast4090 port 40583: Connection refused


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np

import sys
sys.path.append('..')
import util

device = torch.device('cuda:0')


x = np.load('./bin/output/2d_segmentation.npy')

In [3]:
class FilterNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # assert len(layers) >= 2, "layers must have at least 2 elements"

        kernel_size = 9
        layers = [2, 2, 2, 2, 2, 2]

        c = layers[0]
        # L = []
        self.layers = nn.ModuleList()
        for i, l in enumerate(layers[1:-1]):
            # L.append(util.FFTConv3d(c, l, kernel_size, padding=0 if i < 2 else kernel_size//2))
            # L.append(torch.nn.BatchNorm3d(l))
            # L.append(torch.nn.GELU())
            # c = l
            self.layers.append(nn.Sequential(
                util.FFTConv3d(c, 1, kernel_size, padding=0 if i < 2 else kernel_size//2),
                # torch.nn.BatchNorm3d(l),
                torch.nn.GELU(),
            ))
            c = l
        self.layers.append(
            util.FFTConv3d(c, layers[-1], kernel_size, padding=0 if i < 2 else kernel_size//2),
            # torch.nn.BatchNorm3d(layers[-1]),
        )
    
        # self.net = torch.nn.Sequential(*L)
    
    def forward(self, raw_p: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
        # z = self.layers[0](x)
        # z = self.layers[1](z) + (x[..., 32:-32, 32:-32, 32:-32])
        # z = self.layers[2](z)
        # z = self.layers[3](z) + z
        # z = self.layers[4](z) + (x[:, 1:, 32:-32, 32:-32, 32:-32] - 0.5)
        for layer in self.layers:
            z = layer(x)
        return z

# net = util.FFTConv3d(1, 2, 33, padding=0).to(device)
net = FilterNet().to(device)

x = torch.randn(16, 1, patch_size, patch_size, patch_size, device=device)
y = net(x)
print(y.std()), print(y.shape), print(y),

tensor(1.1921, device='cuda:0', grad_fn=<StdBackward0>)
torch.Size([16, 1, 64, 64, 64])
tensor([[[[[-5.8172e-02, -4.7484e-01,  6.3470e-01,  ..., -1.1377e+00,
             4.5077e-01,  6.8499e-01],
           [ 3.1330e+00, -1.6305e+00,  2.4378e-01,  ..., -1.9667e+00,
             2.4382e-01,  2.1385e+00],
           [ 4.1862e-02,  1.0999e+00, -3.8245e-01,  ..., -8.1875e-02,
             7.0443e-01, -1.4601e+00],
           ...,
           [ 1.4998e+00,  1.2391e+00, -3.0259e-01,  ..., -6.4113e-01,
             8.9616e-01,  3.3627e-02],
           [-5.9406e-01,  5.8859e-01,  3.9793e-01,  ..., -8.4901e-01,
            -5.2162e-01, -1.1061e+00],
           [-1.3230e+00, -1.3318e+00, -7.0789e-02,  ..., -2.5801e-01,
            -1.7138e+00,  6.2539e-01]],

          [[-1.9305e+00, -5.8353e-01, -2.6132e-01,  ..., -2.5271e-01,
            -1.0032e+00,  6.3846e-01],
           [-1.0397e+00, -1.7275e+00,  8.0357e-01,  ..., -1.9590e+00,
             8.9784e-01, -4.6311e-01],
           [ 1.3538e+0

(None, None, None)

In [6]:
import torch
from torch.utils.data import DataLoader
import numpy as np

import importlib
import sys
sys.path.append('..')
import util
importlib.reload(util)
importlib.reload(util.general)
importlib.reload(util.data)
importlib.reload(util.nn)
importlib.reload(util.nn.unet3d)
import util
import lutil
importlib.reload(lutil)
import lutil

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
log_board = util.diagnostics.LogBoard('log_dir', 6005)
log_board.launch()

In [8]:
m = lutil.ACDC(320, 320).to(device)
m2 = torch.nn.Conv3d(320, 320, 3, padding=1).to(device)

x = torch.randn(1, 320, 96, 96, 96).to(device)
print(x[0, 0, 0, 0, 0])

In [ ]:
y = m2(x)
x[0, 0, 0, 0, 0], y[0, 0, 0, 0, 0]

(tensor(-1.5430, device='cuda:0'),
 tensor(0.0704+0.0055j, device='cuda:0', grad_fn=<SelectBackward0>))

In [ ]:
with torch.no_grad():
    out = util.diagnostics.do_analysis(
        lambda: m(x),
        extra_stats={}
    )
out

STAGE:2024-01-02 22:06:10 71950:71950 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-01-02 22:06:10 71950:71950 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-01-02 22:06:10 71950:71950 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


{'time_ns': 195203,
 'time_var': 84794377.60640138,
 'flops_mean': 24980781.20588235,
 'flops_var': 1.1163372024956018e+16,
 'mem_mb': 5413.5,
 'profiler': <torch.profiler.profiler.profile at 0x7f99a1b18550>}

In [ ]:
m(x).shape

torch.Size([1, 320, 96, 96, 96])

In [ ]:
m.A.view(-1, m.A.shape[1], *([1] * len(x.shape))).shape

torch.Size([1, 1024, 1, 1, 1, 1])

In [ ]:
m.A.expand(x.shape).shape, x.shape

(torch.Size([1, 1024]), torch.Size([1, 1024, 6, 6]))